In [3]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *
from snowflake.snowpark.functions import udtf

import configparser
import os

In [31]:
def get_snowpark_session():
    ##Get configs
    config = configparser.ConfigParser()
    conf_path = os.path.join('C:\\Users\\Esli\\.snowsql','config')
    config.read(conf_path)

    #Snowflake config
    sfAccount = config['connections.dev']['accountname']
    sfUser = config['connections.dev']['username']
    sfPass = config['connections.dev']['password']
    SfRole = config['connections.dev']['rolename']
    sfWarehouse = config['connections.dev']['warehousename']
    sfDatabase = config['connections.dev']['dbname']
    CONNECTION_PARAMETERS = {
        "account": sfAccount,
        "user": sfUser,
        "password": sfPass,
        "database": sfDatabase,
        "warehouse": sfWarehouse,
        "role": SfRole
    }

    return Session.builder.configs(CONNECTION_PARAMETERS).create()
